In [1]:
# ------------------------------
# DATA CLEANING

# This script cleans the data from the project and outputs a csv file with the cleaned and aggregated data.
# ------------------------------

# ------------------------------
# 1. IMPORTING REQUIRED LIBRARIES
# ------------------------------
import pandas as pd
import numpy as np

# ------------------------------
# 2. DATA IMPORT AND MERGING
# ------------------------------
# Import data from csv files
df_dems = pd.read_csv('../data/raw_data/prolific_dem.csv')
df_repubs = pd.read_csv('../data/raw_data/prolific_repub.csv')
df_repubs_lucid = pd.read_csv('../data/raw_data/lucid_repub.csv')

# Add 'party' column to identify the origin of the data
df_dems['party'] = 'dem'
df_repubs['party'] = 'repub'
df_repubs_lucid['party'] = 'repub'

#rename ResponseId column in df_repubs_lucid to prolific_ID
df_repubs_lucid.rename(columns={'ResponseId':'prolific_ID'}, inplace=True)

# Merge the dataframes vertically
df = pd.concat([df_dems, df_repubs, df_repubs_lucid], axis=0)

# ------------------------------
# 3. DATA CLEANING AND TRANSFORMATION
# ------------------------------
# Merge control and non-control dv response columns and remove the old control columns
column_pairs = [
    ('control_vax_1_1', 'vax_1_1'),
    ('control_vax_2_1', 'vax_2_1'),
    ('control_vax_3_1', 'vax_3_1'),
    ('control_vax_4_1', 'vax_4_1'),
    ('control_elec_1_1', 'elec_1_1'),
    ('control_elec_2_1', 'elec_2_1'),
    ('control_elec_3_1', 'elec_3_1'),
    ('control_elec_4_1', 'elec_4_1'),
    ('control_dep_1_1', 'dep_1_1'),
    ('control_dep_2_1', 'dep_2_1'),
    ('control_dep_3_1', 'dep_3_1'),
    ('control_dep_4_1', 'dep_4_1'),
]

df = df.reset_index(drop=True)

for control_col, non_control_col in column_pairs:
    df[non_control_col] = df[non_control_col].combine_first(df[control_col])

df.drop(columns=[col for col, _ in column_pairs], inplace=True)

# Convert non-numeric values to NaN for dv response columns and then drop all NaN rows
columns_to_check = ['vax_1_1', 'vax_2_1', 'vax_3_1', 'vax_4_1', 'elec_1_1', 'elec_2_1', 'elec_3_1', 'elec_4_1', 'dep_1_1', 'dep_2_1', 'dep_3_1', 'dep_4_1']
for col in columns_to_check:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=columns_to_check, how='any', inplace=True)

# Remove other misc rows based on NaN conditions for authorship and attention check questions
df = df[~(df['attention_check_1'].isna() & df['attention_check_2'].isna())]
df = df[~(df['authorship'].isna())]

# Create new 'condition' column containing both treatment and stance info
df.rename(columns={'condition':'treatment'}, inplace=True)
df['condition'] = df['treatment'] + '_' + df['stance']
df['condition'] = df['condition'].replace('control_na', 'control')

# Drop rows where prolific_ID is null
df = df[df['prolific_ID'].notna()]

# ------------------------------
# 4. FEATURE ENGINEERING
# ------------------------------
# Reverse score 2nd item for each dv question battery
df['vax_2_1'] = 100 - df['vax_2_1']
df['elec_2_1'] = 100 - df['elec_2_1']
df['dep_2_1'] = 100 - df['dep_2_1']

# Calculate mean scores for each dv question battery
dv_response_mean_cols = {
    'vax': ['vax_1_1', 'vax_2_1', 'vax_3_1', 'vax_4_1'],
    'elec': ['elec_1_1', 'elec_2_1', 'elec_3_1', 'elec_4_1'],
    'dep': ['dep_1_1', 'dep_2_1', 'dep_3_1', 'dep_4_1']
}
for key, cols in dv_response_mean_cols.items():
    df[f'dv_response_mean_{key}'] = df[cols].mean(axis=1)
    
# Create alignment variable where 1 = aligned and 0 = not aligned
df['aligned'] = np.where(((df['treatment'] == 'repub') & (df['party'] == 'repub')) | 
                            ((df['treatment'] == 'dem') & (df['party'] == 'dem')), 1, 0)
    
# ------------------------------
# 5. COLUMN REARRANGEMENT AND RENAMING
# ------------------------------
# Specify the order of columns and filter the dataframe to drop unneeded columns
columns_to_keep = ['prolific_ID', 'party', 'pol_affil', 'age', 'gender', 'ethnicity', 'StartDate', 'EndDate', 'attention_check_1', 'attention_check_2', 'treatment', 'stance', 'condition', 'aligned', 'vax_1_1', 'vax_2_1', 'vax_3_1', 'vax_4_1', 'elec_1_1', 'elec_2_1', 'elec_3_1', 'elec_4_1', 'dep_1_1', 'dep_2_1', 'dep_3_1', 'dep_4_1', 'dv_response_mean_vax', 'dv_response_mean_elec', 'dv_response_mean_dep', 'vax_partisanship', 'elec_partisanship', 'dep_partisanship', 'authorship']
df = df[columns_to_keep]

# Rename columns for clarity
df.rename(columns={'vax_1_1':'vax_1', 'vax_2_1':'vax_2', 'vax_3_1':'vax_3', 'vax_4_1':'vax_4', 'elec_1_1':'elec_1', 'elec_2_1':'elec_2', 'elec_3_1':'elec_3', 'elec_4_1':'elec_4', 'dep_1_1':'dep_1', 'dep_2_1':'dep_2', 'dep_3_1':'dep_3', 'dep_4_1':'dep_4'}, inplace=True)

# ------------------------------
# 6. EXPORT CLEANED DATA
# ------------------------------
df.to_csv('roleplay_cleaned.csv', index=False)

In [2]:
# Print the number of dems and repubs in the cleaned data
print(df['party'].value_counts())

party
repub    2501
dem      2454
Name: count, dtype: int64
